In [1]:
# -*- coding: UTF-8 -*-
import time
from datetime import datetime
from datetime import timedelta
import pandas as pd
import pickle
import os
import math
import numpy as np

In [2]:
train = "pre/train.csv"
ad = "pre/ad.csv"
app_categories = "pre/app_categories.csv"
position = "pre/position.csv"
user_app_actions = "pre/user_app_actions.csv"
user_installedapps = "pre/user_installedapps.csv"
user = "pre/user.csv"
test = "pre/test.csv"

In [8]:
print len(d[d.label == 1])
print len(d[d.label == 0])

93262
3656266


In [3]:
#获取用户基本信息
def homedown1(x):
    x = str(x)
    if(len(x)==1):
        return -1
    elif(len(x)==3):
        return x[0]
    else:
        return x[0]+x[1]

def homedown2(x):
    x = str(x)
    if(len(x)==1):
        return -1
    elif(len(x)==3):
        return x[1]+x[2]
    else:
        return x[2]+x[3]

def get_user(data):

    data = pd.read_csv(data)
    fun1 = lambda x:-1 if(x==0) else (int)((x-1)/10)
    
    age = pd.get_dummies(data['age'].map(fun1),prefix='age')

    gender = pd.get_dummies(data['gender'],prefix='gender')

    education = pd.get_dummies(data['education'], prefix='education')

    marriageStatus = pd.get_dummies(data['marriageStatus'], prefix='marriageStatus')

    haveBaby = pd.get_dummies(data['haveBaby'], prefix='haveBaby')

    hometown_x = pd.get_dummies(data['hometown'].map(homedown1), prefix='hometown_x')

    hometown_y = pd.get_dummies(data['hometown'].map(homedown2), prefix='hometown_y')

    residence_x = pd.get_dummies(data['residence'].map(homedown1), prefix='residence_x')

    residence_y = pd.get_dummies(data['residence'].map(homedown2), prefix='residence_y')

    user_basic = pd.concat([data['userID'], age, gender, education, marriageStatus, haveBaby, hometown_x, 
                            hometown_y, residence_x, residence_y], axis = 1)
    
    return user_basic

In [4]:
#统计用户下载APP个数
def get_user_installedapps(data):
    data = pd.read_csv(data)
    data = data.groupby("userID").size()
    data = data.reset_index()
    data.columns = ["userID", "app_install"]
    
    return data

In [5]:
#获取用户APP行为特征
def get_app_action(end_date, data):
    data = pd.read_csv(data, dtype = {'installTime': np.str})
    data = data[(data.installTime < end_date)]
    data = data.groupby('userID').size()
    data = data.reset_index()
    data.columns = ['userID','app_number']
    
    return data

In [6]:
#统计APP类别
def get_app_categories(data):
    data = pd.read_csv(data)
    fun1 = lambda x: -1 if(x == 0) else (int)(x/100)
    fun2 = lambda x: -1 if(x == 0) else (int)(x/10)+(int)(x%10)

    appCategory1 = pd.get_dummies(data['appCategory'].map(fun1), prefix = 'appCategory1')
    appCategory2 = pd.get_dummies(data['appCategory'].map(fun2), prefix = 'appCategory2')
    appCategory3 = pd.get_dummies(data['appCategory'], prefix = 'appCategory3')
    appCategory = pd.concat([data['appID'], appCategory1, appCategory2, appCategory3], axis = 1)
    
    return appCategory

In [7]:
#统计广告
def get_ad_app(data, data2):
    ad = pd.read_csv(data)
    appPlatform = pd.get_dummies(ad["appPlatform"], prefix = "appPlatform")
    ad = pd.concat([ad, appPlatform], axis = 1)
    app = get_app_categories(data2)
    ad = ad.drop_duplicates('creativeID')
    ad_app = pd.merge(ad, app, how = 'left', on = 'appID')

    return ad_app

In [8]:
#统计用户点击特征
def get_user_ad_click(end_date, data):
    data = pd.read_csv(data, dtype = {'clickTime': np.str})
    data = data[data.clickTime < end_date]
    user_click = data.groupby('userID').size()
    user_click = user_click.reset_index()
    user_click.columns = ['userID', 'user_ad_number']

    user_click_use = data[data.label == 1]
    user_click_use = user_click_use.groupby('userID').size()
    user_click_use = user_click_use.reset_index()
    user_click_use.columns = ['userID', 'user_ad_use']

    res = pd.merge(user_click,user_click_use,how = 'left',on = 'userID')
    res = res.fillna(0)

    fun  = lambda x: -1 if(x[1]==0) else x[0]/x[1]

    res['user_click_ratio'] = res[['user_ad_use','user_ad_number']].apply(fun,axis=1)

    return res

In [9]:
#获取广告位置
def get_position(data):
    data = pd.read_csv(data)
    
    return data

In [2]:
#广告ID和用户信息组合特征
def get_adUser(data):
    fun = lambda x: 1 if (x[0]*x[1] >= 1) else 0
    data["adgender_0"] = data[["adID", "gender_0"]].apply(fun, axis = 1)
    data["adgender_1"] = data[["adID", "gender_1"]].apply(fun, axis = 1)
    data["adgender_2"] = data[["adID", "gender_2"]].apply(fun, axis = 1)
    
    data["adage_-1"] = data[["adID", "age_-1"]].apply(fun, axis = 1)
    data["adage_0"] = data[["adID", "age_0"]].apply(fun, axis = 1)
    data["adage_1"] = data[["adID", "age_1"]].apply(fun, axis = 1)
    data["adage_2"] = data[["adID", "age_2"]].apply(fun, axis = 1)
    data["adage_3"] = data[["adID", "age_3"]].apply(fun, axis = 1)
    data["adage_4"] = data[["adID", "age_4"]].apply(fun, axis = 1)
    data["adage_5"] = data[["adID", "age_5"]].apply(fun, axis = 1)
    data["adage_6"] = data[["adID", "age_6"]].apply(fun, axis = 1)
    data["adage_7"] = data[["adID", "age_7"]].apply(fun, axis = 1)
    
    data["adeducation_0"] = data[["adID", "education_0"]].apply(fun, axis = 1)
    data["adeducation_1"] = data[["adID", "education_1"]].apply(fun, axis = 1)
    data["adeducation_2"] = data[["adID", "education_2"]].apply(fun, axis = 1)
    data["adeducation_3"] = data[["adID", "education_3"]].apply(fun, axis = 1)
    data["adeducation_4"] = data[["adID", "education_4"]].apply(fun, axis = 1)
    data["adeducation_5"] = data[["adID", "education_5"]].apply(fun, axis = 1)
    data["adeducation_6"] = data[["adID", "education_6"]].apply(fun, axis = 1)
    data["adeducation_7"] = data[["adID", "education_7"]].apply(fun, axis = 1)
    
    data["admarriage_0"] = data[["adID", "marriageStatus_0"]].apply(fun, axis = 1)
    data["admarriage_1"] = data[["adID", "marriageStatus_1"]].apply(fun, axis = 1)
    data["admarriage_2"] = data[["adID", "marriageStatus_2"]].apply(fun, axis = 1)    
    data["admarriage_3"] = data[["adID", "marriageStatus_3"]].apply(fun, axis = 1)
 
    data["adbaby_0"] = data[["adID", "haveBaby_0"]].apply(fun, axis = 1)
    data["adbaby_1"] = data[["adID", "haveBaby_1"]].apply(fun, axis = 1)
    data["adbaby_2"] = data[["adID", "haveBaby_2"]].apply(fun, axis = 1)
    data["adbaby_3"] = data[["adID", "haveBaby_3"]].apply(fun, axis = 1)
    data["adbaby_4"] = data[["adID", "haveBaby_4"]].apply(fun, axis = 1)
    data["adbaby_5"] = data[["adID", "haveBaby_5"]].apply(fun, axis = 1)
    data["adbaby_6"] = data[["adID", "haveBaby_6"]].apply(fun, axis = 1)
    
    return data

In [3]:
#位置ID和用户信息组合
def get_posUser(data):
    fun = lambda x: 1 if (x[0]*x[1] >= 1) else 0
    data["posgender_0"] = data[["positionID", "gender_0"]].apply(fun, axis = 1)
    data["posgender_1"] = data[["positionID", "gender_1"]].apply(fun, axis = 1)
    data["posgender_2"] = data[["positionID", "gender_2"]].apply(fun, axis = 1)
    
    data["posage_-1"] = data[["positionID", "age_-1"]].apply(fun, axis = 1)
    data["posage_0"] = data[["positionID", "age_0"]].apply(fun, axis = 1)
    data["posage_1"] = data[["positionID", "age_1"]].apply(fun, axis = 1)
    data["posage_2"] = data[["positionID", "age_2"]].apply(fun, axis = 1)
    data["posage_3"] = data[["positionID", "age_3"]].apply(fun, axis = 1)
    data["posage_4"] = data[["positionID", "age_4"]].apply(fun, axis = 1)
    data["posage_5"] = data[["positionID", "age_5"]].apply(fun, axis = 1)
    data["posage_6"] = data[["positionID", "age_6"]].apply(fun, axis = 1)
    data["posage_7"] = data[["positionID", "age_7"]].apply(fun, axis = 1)
    
    data["poseducation_0"] = data[["positionID", "education_0"]].apply(fun, axis = 1)
    data["poseducation_1"] = data[["positionID", "education_1"]].apply(fun, axis = 1)
    data["poseducation_2"] = data[["positionID", "education_2"]].apply(fun, axis = 1)
    data["poseducation_3"] = data[["positionID", "education_3"]].apply(fun, axis = 1)
    data["poseducation_4"] = data[["positionID", "education_4"]].apply(fun, axis = 1)
    data["poseducation_5"] = data[["positionID", "education_5"]].apply(fun, axis = 1)
    data["poseducation_6"] = data[["positionID", "education_6"]].apply(fun, axis = 1)
    data["poseducation_7"] = data[["positionID", "education_7"]].apply(fun, axis = 1)
    
    data["posmarriage_0"] = data[["positionID", "marriageStatus_0"]].apply(fun, axis = 1)
    data["posmarriage_1"] = data[["positionID", "marriageStatus_1"]].apply(fun, axis = 1)
    data["posmarriage_2"] = data[["positionID", "marriageStatus_2"]].apply(fun, axis = 1)    
    data["posmarriage_3"] = data[["positionID", "marriageStatus_3"]].apply(fun, axis = 1)
 
    data["posbaby_0"] = data[["positionID", "haveBaby_0"]].apply(fun, axis = 1)
    data["posbaby_1"] = data[["positionID", "haveBaby_1"]].apply(fun, axis = 1)
    data["posbaby_2"] = data[["positionID", "haveBaby_2"]].apply(fun, axis = 1)
    data["posbaby_3"] = data[["positionID", "haveBaby_3"]].apply(fun, axis = 1)
    data["posbaby_4"] = data[["positionID", "haveBaby_4"]].apply(fun, axis = 1)
    data["posbaby_5"] = data[["positionID", "haveBaby_5"]].apply(fun, axis = 1)
    data["posbaby_6"] = data[["positionID", "haveBaby_6"]].apply(fun, axis = 1)
    
    return data

In [14]:
#获取特征
#def make_train_set(train_start,train_end):
train_start = '310000'
train_end = '320000'
train_data = pd.read_csv(test, dtype = {'clickTime': np.str})
#del train_data["conversionTime"]
#del train_data["instanceID"]
traindata = train_data[(train_data.clickTime >= train_start) & (train_data.clickTime < train_end)]

user_basic = get_user(user)
appCategory = get_app_categories(app_categories)
ad_app = get_ad_app(ad, app_categories)

user_app_action = get_app_action(train_start, user_app_actions)
user_app_install = get_user_installedapps(user_installedapps)
user_ad_click  = get_user_ad_click(train_start, train)
pos = get_position(position)

print(len(traindata))
traindata = pd.merge(traindata, user_basic, how = 'left', on = 'userID')
print(len(traindata))


print(len(traindata))
traindata = pd.merge(traindata, ad_app, how = 'left', on = 'creativeID')
print(len(traindata))


print(len(traindata))
traindata = pd.merge(traindata, user_app_action, how = 'left', on = 'userID')
print(len(traindata))

print(len(traindata))
traindata = pd.merge(traindata, user_app_install, how = 'left', on = 'userID')
print(len(traindata))

traindata['app_number'] = traindata['app_number'].fillna(0)
traindata['app_install'] = traindata['app_install'].fillna(0)

print(len(traindata))
traindata['app_number'] = traindata['app_number'] + traindata['app_install']
print(len(traindata))

print(len(traindata))
traindata = pd.merge(traindata, user_ad_click, how = 'left', on = 'userID')
print(len(traindata))

traindata['user_ad_number'] = traindata['user_ad_number'].fillna(0)
traindata['user_ad_use'] = traindata['user_ad_use'].fillna(0)
traindata['user_click_ratio'] = traindata['user_click_ratio'].fillna(-1)

traindata = pd.merge(traindata, pos, how = 'left', on = 'positionID')


338489
338489
338489
338489
338489
338489
338489
338489
338489
338489
338489
338489


In [15]:
#生成文件

conType = pd.get_dummies(traindata["connectionType"], prefix = "conType")
teleOperator = pd.get_dummies(traindata["telecomsOperator"], prefix = "teleOperator")
siteID = pd.get_dummies(traindata["sitesetID"], prefix = "siteID")
posType = pd.get_dummies(traindata["positionType"], prefix = "posType")
appPlatform = pd.get_dummies(traindata["appPlatform"], prefix = "appPlatform")

#data2 = pd.concat([traindata["userID"], conType, teleOperator, siteID, posType], axis = 1)
#data1 = pd.merge(traindata, data2, how = "left")

data1 = pd.concat([traindata, conType, teleOperator, siteID, posType, appPlatform], axis = 1)

del data1["connectionType"]
del data1["telecomsOperator"]
del data1["sitesetID"]
del data1["positionType"]
del data1["appPlatform"]
print data1

#data1.to_csv("test31.csv", index = False)

        instanceID  label clickTime  creativeID   userID  positionID  age_-1  \
0                1     -1    310000        3745  1164848        3451       0   
1                2     -1    310000        2284  2127247        1613       0   
2                3     -1    310000        1456  2769125        5510       0   
3                4     -1    310000        4565     9762        4113       0   
4                5     -1    310000          49  2513636        3615       0   
5                6     -1    310000        3824   488035        3821       0   
6                7     -1    310000         863   225561        4188       0   
7                8     -1    310000        4469  1144620        4113       0   
8                9     -1    310000        3745  1113275        3347       0   
9               10     -1    310000        4565  1361729        2426       0   
10              11     -1    310000        4250  1749379        4292       1   
11              12     -1    310000     

In [16]:
data1.to_csv("test_3032.csv", index = False)